In [1]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

# NASA Mars News Scrape

In [3]:
browser = init_browser()
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
news_html = browser.html
soup = BeautifulSoup(news_html, 'html.parser')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Nick\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [4]:
# retreive news title & news paragraph text
news_div = soup.find_all('div', class_='content_title')
for div in news_div[:2]:
    try:
        news_title = div.find('a').text
    except:
        pass
news_p = soup.find('div', class_='article_teaser_body').text

# JPL Mars Space Images Scrape

In [5]:
# set up splinter with url for scraping
space_img_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
img_url_prefix = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
browser.visit(space_img_url)
space_img_html = browser.html
soup = BeautifulSoup(space_img_html, 'html.parser')

In [6]:
# find featured image
img_result = soup.find('img',class_='headerimage')
# attach full url to image & print link
feat_image_url = img_url_prefix + img_result['src']

# Mars Facts Table Scrape to HTML string

In [14]:
# Mars Facts Table Scrape to HTML string
table_url = 'https://space-facts.com/mars/'
mars_table_html = pd.read_html(table_url, attrs={'id':'tablepress-p-mars-no-2'})

# Mars table to HTML string
mars_df = mars_table_html[0]
mars_df = mars_df.rename(columns={0:"",1:"Mars"})
mars_df.reset_index(drop=True, inplace=True)
mars_df_html = mars_df.to_html(index=False)

In [8]:
mars_html_split = mars_df_html.split("\n")

In [13]:
mars_html = [t for t in mars_html_split]
mars_html[0]='<table class=table-striped class="dataframe">'
glue = "\n"
mars_html = glue.join(mars_html)

# Mars Hemispheres

In [10]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)
time.sleep(2)
hem_html = browser.html
hem_soup = BeautifulSoup(hem_html,'html.parser')

In [11]:
hem_img_urls = []
hem_divs = hem_soup.find_all('div',class_='description')
for div in hem_divs:
    img_title = div.find('a').text
    try:
        browser.click_link_by_partial_text(img_title)
        time.sleep(1)
        browser.click_link_by_text('Sample')
    except:
        print('not found')
    # Grab new window url
    new_url = browser.windows[1].url
    # Close new window
    browser.windows[1].close()
    # Visit new url
    browser.visit(new_url)
    # Grab html and image src url
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    img_tag = soup.find('img')
    full_img = img_tag['src']
    # save url to list
    split = img_title.split(" ", -1)
    del split[-1]
    img_title = (" ").join(split)
    hem_dict = {
        'title': img_title,
        'img_url': full_img
    }
    hem_img_urls.append(hem_dict)
    # Go back to page with mars image list
    browser.back()
    browser.back()


C:\Users\Nick\anaconda3\envs\pythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(
C:\Users\Nick\anaconda3\envs\pythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:498: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  warnings.warn(


In [12]:
browser.quit()